In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
import yaml
import random
import json
from copy import copy
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs

from collections import defaultdict
from probis import *
from run import *

In [3]:
with open("../cfg.yaml", "r") as f:
    cfg = yaml.safe_load(f)

In [4]:
activities_unfiltered = get_crossdocked_chembl_activities(cfg)

Using cached data from get_crossdocked_chembl_activities


In [127]:
class timeout:
    def __init__(self, seconds, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message
    def handle_timeout(self, signum, frame):
        raise TimeoutError(self.error_message)
    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)
    def __exit__(self, type, value, traceback):
        signal.alarm(0)

In [142]:
for (smiles, uniprot), rows in tqdm(dup_rows.items()):
    st_types = [ r.standard_type for r in rows ]
    pchembl_values = [ r.pchembl_value for r in rows ]

    # find the acitvities of k nearest neighbors to the chemical in question
    sim_cutoff = 0.5
    k = 5

    # janky rlu cache
    if len(uniprot2df) > 5:
        # first added uniprot, since dicts preserve addition order
        key = next(iter(uniprot2df))
        del uniprot2df[key]

    if uniprot in uniprot2df:
        df = uniprot2df[uniprot]
    else:
        df = copy(activities_unfiltered.query("protein_accession == @uniprot")).reset_index(drop=True)
        PandasTools.AddMoleculeColumnToFrame(df,smilesCol='canonical_smiles')
        mfp2_fps = rdFingerprintGenerator.GetFPs(list(df['ROMol']))
        df['fp'] = mfp2_fps
        query_fp = next(iter(df.query("canonical_smiles == @smiles").fp))
        similarity = DataStructs.BulkTanimotoSimilarity(query_fp ,df['fp'])
        df["similarity"] = similarity
        uniprot2df[uniprot] = df

    near_pchembls = []
    near_sims = []
    for i, row in df.sort_values(by='similarity', ascending=False).iterrows():
        if row.canonical_smiles == smiles: continue
        if row.similarity < sim_cutoff: break
        if len(near_pchembls) == k: break
        near_pchembls.append(row.pchembl_value)
        near_sims.append(row.similarity)

    near_sims = np.array(near_sims)
    near_pchembls = np.array(near_pchembls)

    # if there's at least 1 found near activitiy, use the value closest
    if len(near_pchembls) > 0:
        # weighted averge
        weights = np.exp(near_sims)/sum(np.exp(near_sims))
        w_mean = sum(weights*near_pchembls)
        final_pchembl = None
        final_st_type = None
        for pchembl, st_type in zip(pchembl_values, st_types):
            if final_pchembl is None or abs(pchembl - w_mean) < abs(final_pchembl - w_mean):
                final_pchembl = pchembl
                final_st_type = st_type

    print(final_pchembl)
    break

  0%|          | 0/160456 [00:00<?, ?it/s]

5.52
